# Exploratory Data Analysis and Basic Imputation of Missing Values

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import sklearn

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model as lm
from sklearn import model_selection as ms
sklearn.set_config(print_changed_only=False)

import xgboost

import copy

import import_ipynb
from _Self_Written_Functions_Sheet import rmse

importing Jupyter notebook from _Self_Written_Functions_Sheet.ipynb


# Data Exploration

In [2]:
# importing the datasets
hp_train = pd.read_csv('..\hp_2a_ranked_edited_train.csv', index_col=0)
hp_test = pd.read_csv('..\hp_2a_ranked_edited_test.csv', index_col=0)
hp_logsaleprice = pd.read_csv('..\hp_1a_no_imputation_logsaleprice.csv', index_col=0)
hp_saleprice = pd.read_csv('..\hp_1a_no_imputation_saleprice.csv')

# saving train index
hp_index = list(hp_train.index)

combo_nonedit = pd.DataFrame(np.concatenate([hp_train, hp_test]))
combo_nonedit.columns = hp_train.columns

for col in combo_nonedit.columns: 
    combo_nonedit[col] = combo_nonedit[col].astype('float', errors='ignore')

combo_cols = list(combo_nonedit.columns)

combo = copy.deepcopy(combo_nonedit)

In [3]:
combo

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,2-STORY 1946 & NEWER,RL,65.0,8450.0,1.0,0.0,Reg,Lvl,3.0,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7.0,5.0,2003.0,2003.0,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,4.0,3.0,PConc,4.0,3.0,1.0,6.0,706.0,1.0,0.0,150.0,856.0,GasA,5.0,1.0,5.0,856.0,854.0,0.0,1710.0,1.0,0.0,2.0,1.0,3.0,1.0,4.0,8.0,7.0,0.0,None,Attchd,2003.0,2.0,2.0,548.0,3.0,3.0,3.0,0.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,2.0,2008.0,WD,Normal
1,1-STORY 1946 & NEWER ALL STYLES,RL,80.0,9600.0,1.0,0.0,Reg,Lvl,3.0,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6.0,8.0,1976.0,1976.0,Gable,CompShg,MetalSd,MetalSd,None,0.0,3.0,3.0,CBlock,4.0,3.0,4.0,5.0,978.0,1.0,0.0,284.0,1262.0,GasA,5.0,1.0,5.0,1262.0,0.0,0.0,1262.0,0.0,1.0,2.0,0.0,3.0,1.0,3.0,6.0,7.0,1.0,TA,Attchd,1976.0,2.0,2.0,460.0,3.0,3.0,3.0,298.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,5.0,2007.0,WD,Normal
2,2-STORY 1946 & NEWER,RL,68.0,11250.0,1.0,0.0,IR1,Lvl,3.0,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7.0,5.0,2001.0,2002.0,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,4.0,3.0,PConc,4.0,3.0,2.0,6.0,486.0,1.0,0.0,434.0,920.0,GasA,5.0,1.0,5.0,920.0,866.0,0.0,1786.0,1.0,0.0,2.0,1.0,3.0,1.0,4.0,6.0,7.0,1.0,TA,Attchd,2001.0,2.0,2.0,608.0,3.0,3.0,3.0,0.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,9.0,2008.0,WD,Normal
3,2-STORY 1945 & OLDER,RL,60.0,9550.0,1.0,0.0,IR1,Lvl,3.0,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7.0,5.0,1915.0,1970.0,Gable,CompShg,Wd Sdng,WdShing,None,0.0,3.0,3.0,BrkTil,3.0,4.0,1.0,5.0,216.0,1.0,0.0,540.0,756.0,GasA,4.0,1.0,5.0,961.0,756.0,0.0,1717.0,1.0,0.0,1.0,0.0,3.0,1.0,4.0,7.0,7.0,1.0,Gd,Detchd,1998.0,1.0,3.0,642.0,3.0,3.0,3.0,0.0,35.0,272.0,0.0,0.0,0.0,0.0,0.0,None,0.0,2.0,2006.0,WD,Abnorml
4,2-STORY 1946 & NEWER,RL,84.0,14260.0,1.0,0.0,IR1,Lvl,3.0,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8.0,5.0,2000.0,2000.0,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,4.0,3.0,PConc,4.0,3.0,3.0,6.0,655.0,1.0,0.0,490.0,1145.0,GasA,5.0,1.0,5.0,1145.0,1053.0,0.0,2198.0,1.0,0.0,2.0,1.0,4.0,1.0,4.0,9.0,7.0,1.0,TA,Attchd,2000.0,2.0,3.0,836.0,3.0,3.0,3.0,192.0,84.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,12.0,2008.0,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2912,2-STORY PUD - 1946 & NEWER,RM,21.0,1936.0,1.0,0.0,Reg,Lvl,3.0,Inside,Gtl,MeadowV,Norm,Norm,TwnhsI,2Story,4.0,7.0,1970.0,1970.0,Gable,CompShg,CemntBd,CemntBd,None,0.0,3.0,3.0,CBlock,3.0,3.0,1.0,1.0,0.0,1.0,0.0,546.0,546.0,GasA,4.0,1.0,5.0,546.0,546.0,0.0,1092.0,0.0,0.0,1.0,1.0,3.0,1.0,3.0,5.0,7.0,0.0,None,None,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,6.0,2006.0,WD,Normal
2913,2-STORY PUD - 1946 & NEWER,RM,21.0,1894.0,1.0,0.0,Reg,Lvl,3.0,Inside,Gtl,MeadowV,Norm,Norm,TwnhsE,2Story,4.0,5.0,1970.0,1970.0,Gable,CompShg,CemntBd,CemntBd,None,0.0,3.0,3.0,CBlock,3.0,3.0,1.0,3.0,252.0,1.0,0.0,294.0,546.0,GasA,3.0,1.0,5.0,546.0,546.0,0.0,1092.0,0.0,0.0,1.0,1.0,3.0,1.0,3.0,6.0,7.0,0.0,None,CarPort,1970.0,1.0,1.0,286.0,3.0,3.0,3.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0

### Creating functions to use as combinations for trials for linear and tree models

In [4]:
# creating a dictionary of the neigborhoods and ranks based on median value
list_neigborhoods = list(combo['Neighborhood'].unique())

n_dict_median = {}

for n in list_neigborhoods:
    n_dict_median[n] = hp_saleprice.loc[hp_train[hp_train['Neighborhood']==n].index].median()[0]

n_dict_median_sort = dict(sorted(n_dict_median.items(), key=lambda item:item[1], reverse=True))

n_list_median_sort = list(n_dict_median_sort.keys())

n_dict_median_ranking = {}
for i, n in enumerate(n_list_median_sort):
    n_dict_median_ranking[n] = i + 1

def conditions_edit():
    # combine Railroad Adjacent and Railroad Within 200 for both railroads and both condition1 and 2
    combo.loc[combo['Condition1'].str.contains('RRN', na=False), 'Condition1'] = 'RRN'
    combo.loc[combo['Condition1'].str.contains('RRA', na=False), 'Condition1'] = 'RRA'

    combo.loc[combo['Condition2'].str.contains('RRN', na=False), 'Condition2'] = 'RRN'
    combo.loc[combo['Condition2'].str.contains('RRA', na=False), 'Condition2'] = 'RRA'

def nbrhd_rank():
    # creating a column replacing the neighborhood name with rank
    combo['NbMedianRank'] = combo['Neighborhood'].replace(n_dict_median_ranking)

def drop_nbrhd():
    combo.drop('Neighborhood', axis=1, inplace = True)

def totallivsf_add():
    # creating a column with total square footage
    combo[['TotalLivSF']] = combo['GrLivArea'] + combo['BsmtFinSF1'] + combo['BsmtFinSF2']

def drop_grlivarea():
    combo.drop(['GrLivArea'], axis=1, inplace = True)

def bsmtfin_add():
    # adding the column for whether basement is finished
    combo['BsmtFin'] = combo['BsmtFinSF1'] + combo['BsmtFinSF2']
    combo.loc[combo['BsmtFin'] > 0, 'BsmtFin'] = 1 

def drop_bsmtfinsf12():
    combo.drop(['BsmtFinSF1', 'BsmtFinSF2'], axis=1, inplace = True)

def totalporchsf_add():
    # adding the total SF of porches
    combo['TotalPorchSF'] = combo['OpenPorchSF'] + combo['EnclosedPorch'] +\
    combo['3SsnPorch'] + combo['ScreenPorch']

def drop_porches():
    combo.drop(['OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch'], axis=1, inplace = True)

def yrsremodtosold_add():
    # calculating the time between remodeling and selling the house
    combo['YrsRemodToSold'] = combo['YrSold'] - combo['YearRemodAdd']

def drop_yrsoldremod():
    combo.drop(['YrSold', 'YearRemodAdd'], axis=1, inplace = True)

def fireplace_yes():
    # Replacing number of fireplaces with either yes (1) or no (0)
    combo['Fireplace'] = [1 if x > 0 else 0 for x in combo['Fireplaces']]

def drop_fireplaces():
    combo.drop(['Fireplaces'], axis=1, inplace = True)

def pool_yes(): 
    # Replacing pool area with either yes (1) or no (0)
    combo['Pool'] = [1 if x > 0 else 0 for x in combo['PoolArea']]

def drop_poolarea():
    combo.drop(['PoolArea'], axis=1, inplace = True)

def totalbaths_add():
    # adding the number of bathrooms
    combo['TotalBaths'] = combo['FullBath'] + combo['HalfBath'] * 0.5

def drop_baths():
    combo.drop(['FullBath', 'HalfBath'], axis=1, inplace = True)
    
def totalbsmtbaths_add():
    # adding the number of bathrooms
    combo['TotalBsmtBaths'] = combo['BsmtFullBath'] + combo['BsmtHalfBath'] * 0.5
    
def drop_bsmtbaths():
    combo.drop(['FullBath', 'HalfBath'], axis=1, inplace = True)

### Creating functions from the models to call them below

In [244]:
def mlr_model(x_trn, y_trn, x_tst, y_tst):
    lm = LinearRegression()
    # parameters to use
    normilize = [True, False]
    fitintercept = [True, False]
    gparam_mlr_1 = {'fit_intercept': fitintercept, 'normalize': normilize}
    # setting parameters
    gs_mlr_1 = ms.GridSearchCV(lm, gparam_mlr_1, cv=n_folds, refit=True, n_jobs=-1,
                          scoring='neg_root_mean_squared_error', return_train_score=True)
    gs_mlr_1.fit(x_trn, y_trn)
    mlr_model.test_rmse_mlr = rmse(gs_mlr_1, y_tst, x_tst)    
    
def lasso_model(x_trn, y_trn, x_tst, y_tst):
    lasso = Lasso(random_state=state, max_iter=1000000, selection='random',
              normalize=True, fit_intercept=True)
    # parameters to use
    alphas = np.linspace(1e-4, 2.5e-4, 30)
    gparam_lasso_1 = {'alpha': alphas}
    # setting parameters
    gs_lasso_1 = ms.GridSearchCV(lasso, gparam_lasso_1, cv=n_folds, refit=True, n_jobs=-1,
                          scoring='neg_root_mean_squared_error', return_train_score=True)
    gs_lasso_1.fit(x_trn, y_trn)
    lasso_model.test_rmse_lasso = rmse(gs_lasso_1, y_tst, x_tst)

def ridge_model(x_trn, y_trn, x_tst, y_tst):
    ridge = Ridge(random_state=state, max_iter=10000, fit_intercept=True)
    # parameters to use
    solvers = ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs']
    fitintercept = [True, False]
    gparam_ridge_1 = {'solver': solvers,
                      'fit_intercept': fitintercept, 
                      'random_state': [state]}
    # setting parameters
    gs_ridge_1 = ms.GridSearchCV(ridge, gparam_ridge_1, cv=n_folds, refit=True, n_jobs=-1,
                          scoring='neg_root_mean_squared_error', return_train_score=True)
    gs_ridge_1.fit(x_trn, y_trn)
    ridge_model.test_rmse_ridge = rmse(gs_ridge_1, y_tst, x_tst)
    
def enet_model(x_trn, y_trn, x_tst, y_tst):
    enet = ElasticNet(random_state=state, max_iter=10000, normalize=True, fit_intercept=True)
    # parameters to use
    alphas = [1e-4, 1e-3, 1e-2, 1e-1, 0, 1, 10, 100]
    l1ratio = [0, 0.25, 0.5, 0.75, 1]
    fitintercept = [True, False]
    precomputes = [True, False]
    warmstart = [True, False]
    selections = ['cyclic', 'random']
    gparam_enet_1 = {'alpha': alphas,
                      'l1_ratio': l1ratio,
                      'fit_intercept': fitintercept,
                      'precompute': precomputes, 
                      'warm_start': warmstart,
                      'selection': selections,
                      'random_state': [state]}
    # setting parameters
    gs_enet_1 = ms.GridSearchCV(enet, gparam_enet_1, cv=n_folds, refit=True, n_jobs=-1,
                          scoring='neg_root_mean_squared_error', return_train_score=True)
    gs_enet_1.fit(x_trn, y_trn)
    enet_model.test_rmse_enet = rmse(gs_enet_1, y_tst, x_tst)
    
def rfr_model(x_trn, y_trn, x_tst, y_tst):
    rfr = RandomForestRegressor()
    gparam_rfr = {}
    # setting parameters
    gs_rfr = ms.GridSearchCV(rfr, gparam_rfr, cv=n_folds, refit=True, n_jobs=-1,
                               scoring='neg_root_mean_squared_error', return_train_score=True)
    gs_rfr.fit(x_trn, y_trn)
    rfr_model.test_rmse_rfr = rmse(gs_rfr, y_tst, x_tst)
    
def gbm_model(x_trn, y_trn, x_tst, y_tst):
    gbm = GradientBoostingRegressor()
    gparam_gbm = {}
    gs_gbm = GridSearchCV(gbm, gparam_gbm, cv=n_folds, refit=True, n_jobs=-1, 
                          scoring='neg_root_mean_squared_error', return_train_score=True)
    gs_gbm.fit(x_trn, y_trn)
    gbm_model.test_rmse_gbm = rmse(gs_gbm, y_tst, x_tst)
    
def xgb_model(x_trn, y_trn, x_tst, y_tst):
    xgb_t = xgboost.XGBRegressor()
    gparam_xgb_t = {}
    gs_xgb_t = GridSearchCV(xgb_t, gparam_xgb_t, cv=n_folds, refit=True, verbose=1,
                          scoring='neg_root_mean_squared_error', return_train_score=True)
    gs_xgb_t.fit(x_trn, y_trn)
    xgb_model.test_rmse_xgb = rmse(gs_xgb_t, y_tst, x_tst)

In [245]:
def dum_scale():
    # dummify the variables
    hp_dum = pd.get_dummies(combo, drop_first=True)
    enc = OneHotEncoder()
    for feat in ['MoSold', 'YrSold']:
        if feat in hp_dum.columns:
            # encode the feature
            enc.fit(hp_dum[[feat]])
            # create array and then dataframe of the array
            oh_labels = enc.transform(hp_dum[[feat]]).toarray()
            mssubcl_dum = pd.DataFrame(oh_labels)
            mssubcl_dum.columns = enc.get_feature_names([feat])
            # Concatenate the dataframes and drop Id and original MSSubClass
            hp_dum = pd.concat([hp_dum, mssubcl_dum], axis=1, sort=False)
            hp_dum = hp_dum.drop([feat], axis=1)
    hp_dum_cols = list(hp_dum.columns)
    for col in hp_dum_cols:
        hp_dum[col] = MinMaxScaler().fit_transform(np.array(hp_dum[col]).reshape(-1,1))
        hp_dum.columns = hp_dum_cols
    dum_scale.hp_dum = copy.deepcopy(hp_dum)

In [247]:
# getting scores for all models using every possible combination of functions above

state = 0

# using standard kfold split
n_folds = ms.KFold(n_splits=5, random_state=state, shuffle=True)

stuff_add = [nbrhd_rank, totallivsf_add, bsmtfin_add, totalporchsf_add, yrsremodtosold_add,
             fireplace_yes, pool_yes, totalbaths_add, totalbsmtbaths_add, conditions_edit]

stuff_drop = [drop_nbrhd, drop_grlivarea, drop_bsmtfinsf12,  drop_porches, drop_yrsoldremod,
              drop_fireplaces, drop_poolarea, drop_baths, drop_bsmtbaths, conditions_edit]

models = [mlr_model, lasso_model,  enet_model, rfr_model, gbm_model, xgb_model]
# ridge_model,

# numbers will be indices later down
funcs_scores = {1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}}

subset_list = []

import itertools
# import commands
#https://stackoverflow.com/questions/464864/how-to-get-all-possible-combinations-of-a-list-s-elements

for dual_add, dual_drop in zip(stuff_add, stuff_drop):
    for superset in range(0, len(stuff_add)+1):
        for subset in itertools.combinations([dual_add, dual_drop], superset):
            funcs_used_list = str(subset).split()[1::4]
            
            if funcs_used_list not in subset_list:
                combo = copy.deepcopy(combo_nonedit)
                # run the feat eng func
                for subs in subset:
                    # Get name?
    #                 subs.
                    subs()

                for col in combo.columns: 
                    combo[col] = combo[col].astype('float', errors='ignore')

                dum_scale()

                hp = dum_scale.hp_dum.loc[hp_index]

                funcs_used = ', '.join(str(subset).split()[1::4])

                # setting up train and test sets
                xtrain, xtest, ytrain, ytest = ms.train_test_split(hp, hp_logsaleprice,
                                                                   test_size=0.2, random_state=state)
                # Must flatten to fit
                ytrain = ytrain.values.flatten()
                # run the models            

                for i, model in enumerate(models):
                    model(xtrain, ytrain, xtest, ytest)
                    i+=1
                    funcs_scores[i][funcs_used] = model.test_rmse
                subset_list.append(funcs_used_list)
            else:
                continue



                
#             funcs_used = ', '.join(str(subset).split()[1::4])

            
            
#             print('\nfunctions used:', funcs_used, '\n')
#             print('mlr test rmse is:', mlr_model.test_rmse_mlr)
#             print('lasso test rmse is:', lasso_model.test_rmse_lasso)
#             print('enet test rmse is:', enet_model.test_rmse_enet)
#             print('rfr test rmse is:', rfr_model.test_rmse_rfr)
#             print('gbm test rmse is:', gbm_model.test_rmse_gbm)
#             print('xgb test rmse is:', xgb_model.test_rmse_xgb)
#             print('#'*50, '\n')

()


AttributeError: 'function' object has no attribute 'test_rmse_mlr'

In [159]:
def a():
    print('1')
def b():
    print('2')
def c():
    print('3')
def d():
    print('4')
def e():
    print('5')
def f():
    print('6')
def g():
    print('7')
def h():
    print('8')
def i():
    print('9')
def j():
    print('0')
    
def aa():
    print('11')
def bb():
    print('22')
def cc():
    print('33')
def dd():
    print('44')
def ee():
    print('55')
def ff():
    print('66')
def gg():
    print('77')
def hh():
    print('88')
def ii():
    print('99')
def jj():
    print('00')

In [177]:
i=0

stuff_add = [a, b, c, d, e, f, g, h, i, j]

stuff_drop = [aa, bb, cc, dd, ee, ff, gg, hh, ii, jj]


for r_a, r_b in zip(range(len(stuff_add)+1), range(len(stuff_add)+1)):
    com_list_a.append(list(itertools.combinations(stuff_add, r_a))
    com_list_b = list(itertools.combinations(stuff_drop, r_b))
    
# getting all permutations of the 2 lists
for subset in itertools.product(com_list_b, com_list_a):
    if subset == ((), ()):
        funcs_used_list = ['No changes to dataset']
    else:
        funcs_used_list = str(subset).split()[1::4]
    if funcs_used_list not in subset_list:
        for subs in subset:
            for s in subs:
                s()

11
22
33
44
55
66
77
88
99
00
1
2
3
4
5
6
7
8


TypeError: 'int' object is not callable

In [176]:
com_list_a 

[(<function __main__.a()>,),
 (<function __main__.b()>,),
 (<function __main__.c()>,),
 (<function __main__.d()>,),
 (<function __main__.e()>,),
 (<function __main__.f()>,),
 (<function __main__.g()>,),
 (<function __main__.h()>,),
 (0,),
 (<function __main__.j()>,)]

In [189]:
import numpy as np
np.linspace(1e-4, 1.6e-4, 6)

array([0.0001  , 0.000112, 0.000124, 0.000136, 0.000148, 0.00016 ])

In [111]:
callable(a)

True

In [106]:
type(a)

function

In [143]:
import itertools
stuff_add = [a, b, c]
stuff_drop = [d, e, f]

sub_a_list = []
sub_b_list = []
com_list = []

for r_a, r_b in zip(range(len(stuff_add)+1), range(len(stuff_add)+1)):
    com_list_a = list(itertools.combinations(stuff_add, r_a))
    com_list_b = list(itertools.combinations(stuff_drop, r_b))
    for com in itertools.product(com_list_a, com_list_b):
        funcs = str(com).split()[1::4]
        for fun in com:
            for n in fun:
                n()
#             fun()
#             if fun!='())':
#                 print(''.join(fun))
#                 fun()
        
            


1
4
1
5
1
6
2
4
2
5
2
6
3
4
3
5
3
6
1
2
4
5
1
2
4
6
1
2
5
6
1
3
4
5
1
3
4
6
1
3
5
6
2
3
4
5
2
3
4
6
2
3
5
6
1
2
3
4
5
6


In [133]:
import itertools
stuff_add = [a, b, c]
stuff_drop = [d, e, f]

sub_a_list = []
sub_b_list = []
com_list = []

for r_a, r_b in zip(range(len(stuff_add)+1), range(len(stuff_add)+1)):
    com_list_a = list(itertools.combinations(stuff_add, r_a))
    com_list_b = list(itertools.combinations(stuff_drop, r_b))
    for com_b in com_list_b:
        for com_a in com_list_a:
            
            print('###')
        print([com_a, com_b])

            


###
[(), ()]
###
###
###
[(<function c at 0x00000261CB3FC160>,), (<function d at 0x00000261CB3FCE50>,)]
###
###
###
[(<function c at 0x00000261CB3FC160>,), (<function e at 0x00000261CB3FCD30>,)]
###
###
###
[(<function c at 0x00000261CB3FC160>,), (<function f at 0x00000261CB3FCA60>,)]
###
###
###
[(<function b at 0x00000261C5D5EA60>, <function c at 0x00000261CB3FC160>), (<function d at 0x00000261CB3FCE50>, <function e at 0x00000261CB3FCD30>)]
###
###
###
[(<function b at 0x00000261C5D5EA60>, <function c at 0x00000261CB3FC160>), (<function d at 0x00000261CB3FCE50>, <function f at 0x00000261CB3FCA60>)]
###
###
###
[(<function b at 0x00000261C5D5EA60>, <function c at 0x00000261CB3FC160>), (<function e at 0x00000261CB3FCD30>, <function f at 0x00000261CB3FCA60>)]
###
[(<function a at 0x00000261CB82E430>, <function b at 0x00000261C5D5EA60>, <function c at 0x00000261CB3FC160>), (<function d at 0x00000261CB3FCE50>, <function e at 0x00000261CB3FCD30>, <function f at 0x00000261CB3FCA60>)]


In [78]:
list(itertools.product(com_list_a, com_list_b))

[((<function __main__.a()>, <function __main__.b()>, <function __main__.c()>),
  (<function __main__.d()>, <function __main__.e()>, <function __main__.f()>))]

In [54]:
subs = []
for r_a in range(0, len(stuff_add)+1):
    for subset_a, subset_b in zip(itertools.combinations(stuff_add, r_a),
                                  itertools.combinations(stuff_drop, r_a)):
        for subs in zip(subset_a, subset_b):
            print(subs)


([<function a at 0x00000261C9AAA8B0>], [<function g at 0x00000261C9AD61F0>])
([<function b at 0x00000261C9AAAEE0>, <function c at 0x00000261C9AAAF70>], [<function h at 0x00000261C9AD6280>, <function i at 0x00000261C9AD6310>])
([<function a at 0x00000261C9AAA8B0>], [<function g at 0x00000261C9AD61F0>])
([<function b at 0x00000261C9AAAEE0>, <function c at 0x00000261C9AAAF70>], [<function h at 0x00000261C9AD6280>, <function i at 0x00000261C9AD6310>])


In [26]:
for sub in subab:
    for s in sub:
        if s != []:
            print(*s)
        else:
            continue

[<function g at 0x00000261C9AD61F0>]
[<function h at 0x00000261C9AD6280>, <function i at 0x00000261C9AD6310>]
[<function g at 0x00000261C9AD61F0>] [<function h at 0x00000261C9AD6280>, <function i at 0x00000261C9AD6310>]
[<function a at 0x00000261C9AAA8B0>]
[<function a at 0x00000261C9AAA8B0>]
[<function g at 0x00000261C9AD61F0>]
[<function a at 0x00000261C9AAA8B0>]
[<function h at 0x00000261C9AD6280>, <function i at 0x00000261C9AD6310>]
[<function a at 0x00000261C9AAA8B0>]
[<function g at 0x00000261C9AD61F0>] [<function h at 0x00000261C9AD6280>, <function i at 0x00000261C9AD6310>]
[<function b at 0x00000261C9AAAEE0>, <function c at 0x00000261C9AAAF70>]
[<function b at 0x00000261C9AAAEE0>, <function c at 0x00000261C9AAAF70>]
[<function g at 0x00000261C9AD61F0>]
[<function b at 0x00000261C9AAAEE0>, <function c at 0x00000261C9AAAF70>]
[<function h at 0x00000261C9AD6280>, <function i at 0x00000261C9AD6310>]
[<function b at 0x00000261C9AAAEE0>, <function c at 0x00000261C9AAAF70>]
[<function

In [35]:
subab = [list(tup) for tup in itertools.product(list(sub_a_list), list(sub_b_list))
for sub in subab:
    print(sub[0])

TypeError: 'tuple' object is not callable

In [7]:
subab = list(itertools.product(sub_a_list, sub_b_list))
funcs_used_list = str(subab).split()[1::4]
for sub in funcs_used_list:
    print(sub)

[]),
at
h
i
[[<function
[<function
<function
([[<function
[]),
0x00000261C9AAA8B0>]],
0x00000261C9AD61F0>]]),
0x00000261C9AAA8B0>]],
0x00000261C9AD6280>,
0x00000261C9AD6310>]]),
0x00000261C9AAA8B0>]],
0x00000261C9AD61F0>],
0x00000261C9AD6280>,
0x00000261C9AD6310>]]),
0x00000261C9AAAEE0>,
0x00000261C9AAAF70>]],
at
at
at
at
at
at
at
at
at
at
at
at
at
at
at
d
e
f
g
d
e
f
h
i
d
e
f
g
h
i
a
b
c
([[<function
[<function
<function
[[<function
([[<function
[<function
<function
[[<function
<function
([[<function
[<function
<function
[[<function
[<function
<function
([[<function
[<function
<function
<function
[]),
0x00000261C9AAA8B0>],
0x00000261C9AD6040>,
0x00000261C9AD60D0>,
0x00000261C9AD6160>]],
0x00000261C9AD61F0>]]),
0x00000261C9AAA8B0>],
0x00000261C9AD6040>,
0x00000261C9AD60D0>,
0x00000261C9AD6160>]],
0x00000261C9AD6280>,
0x00000261C9AD6310>]]),
0x00000261C9AAA8B0>],
0x00000261C9AD6040>,
0x00000261C9AD60D0>,
0x00000261C9AD6160>]],
0x00000261C9AD61F0>],
0x00000261C9AD6280>,
0x00000261C9AD63

In [39]:
for r in range(0, len(subseta)+1):
    for subsetab in itertools.combinations(alist, r):
        print([subseta])

[(['a'], ['b', 'c'], ['d', 'e', 'f'])]
[(['a'], ['b', 'c'], ['d', 'e', 'f'])]
[(['a'], ['b', 'c'], ['d', 'e', 'f'])]
[(['a'], ['b', 'c'], ['d', 'e', 'f'])]
[(['a'], ['b', 'c'], ['d', 'e', 'f'])]
[(['a'], ['b', 'c'], ['d', 'e', 'f'])]
[(['a'], ['b', 'c'], ['d', 'e', 'f'])]
[(['a'], ['b', 'c'], ['d', 'e', 'f'])]


In [ ]:
alist = ['a', 'b']
dict_dict = {1: {}, 2:{}}
for i, x in enumerate(alist):
    i+=1
    if dict_dict[i] == {}:
        dict_dict[i]

In [25]:
dict_dict = {1: {'a':5, 'b': 3, 'c': 8}, 2:{'d': 7, 'e': 2, 'f':1}}

min(dict_dict[1], key=dict_dict[1].get)

'b'

In [30]:
for k_1 in dict_dict:
    k_min = min(dict_dict[k_1], key=dict_dict[k_1].get)
    v_min = dict_dict[k_1][k_min]
    print(k_1)
    print(k_min)
    print(v_min)
    print('#' * 50)

1
b
3
##################################################
2
f
1
##################################################


In [22]:
dict_dict

{2: {}}

In [23]:
def a():
    pass
def b():
    pass

ab_list = [a, b]
print(str(ab_list).split()[1::4])

['a', 'b']


In [6]:
# saving the imputed AND dummified AND scaled datasets
hp_train_full.to_csv('..\hp_3a_train_tree.csv')
hp_test_full.to_csv('..\hp_3a_test_tree.csv')

# For Linear

In [8]:
# dummify the variables
hp_dum = pd.get_dummies(combo, drop_first=True)

# scaling the dataset
from sklearn.preprocessing import MinMaxScaler
import copy

hp_dum_cols = list(hp_dum.columns)
hp_full = copy.deepcopy(hp_dum)

for col in hp_dum_cols:
    hp_full[col] = MinMaxScaler().fit_transform(np.array(hp_full[col]).reshape(-1,1))
    
hp_train_full = hp_full.loc[hp_index]
hp_test_full = hp_full.loc[1458:]

# saving the imputed AND dummified AND scaled datasets
hp_train_full.to_csv('..\hp_3a_train_linear.csv')
hp_test_full.to_csv('..\hp_3a_test_linear.csv')

In [9]:
# # converting columns containing numbers to numeric type
# for col in combo.columns: 
#     combo[col] = combo[col].astype('float', errors='ignore')
    
# combo.info()